In [2]:
from gensim.models import Doc2Vec as D2V
from gensim.models.doc2vec import TaggedDocument
from nltk import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, join
from nltk import download as nltkDownload

In [43]:
from db_helper import DBHelper
import re

In [4]:
nltkDownload('stopwords')

[nltk_data] Downloading package stopwords to /home/hmenn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
db = DBHelper()
db.connect(user="root",passwd="Hasan5695*",db="cse496")
tweets = db.getTweets("denemeShort")
tweets.sort(key=lambda t: t.date)

In [6]:
print("ExampleSentName:",tweets[0].text)

ExampleSentName: Hayat bu, bir bakarsın herşey bir anda son bulur.
 Hayat bu, son dediğin an herşey yeniden can bulur..Şemsi Tebrizi


In [7]:
tokenizer = RegexpTokenizer(r'\w+')
stopword_set = set(stopwords.words("turkish"))

In [49]:
# clear tweets
for tweet in tweets:
    tweet.text = re.sub(r"RT(^|[^@\w])@(\w{1,15})\b:","",tweet.text)
print(tweets[2].text)

 " Hiçbir su, zalimlerin yüzkarasını temizleyemez"


In [8]:
def nlp_clean(data):
    new_data = []
    for d in data:
        new_str = d.lower()
        dlist = tokenizer.tokenize(new_str)
        dlist = list(set(dlist).difference(stopword_set))
        new_data.append(dlist)
    return new_data

In [9]:
cleanData = nlp_clean([tweet.text for tweet in tweets])

In [10]:
class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
              yield gensim.models.doc2vec.LabeledSentence(doc,[self.labels_list[idx]])

In [59]:
taggedData = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate([tweet.text for tweet in tweets[:500]])]

In [60]:
print(len(taggedData))

500


In [61]:
max_epochs = 500
vec_size = 20
alpha = 0.025

model = D2V(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.025,
                min_count=1,
                dm =1)
  
model.build_vocab(taggedData)

for epoch in range(max_epochs):
    if epoch %10 == 0:
        print('iteration {0}'.format(epoch))
    model.train(taggedData,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
model.save("../outputs/d2v.model")

iteration 0


/home/hmenn/Workspace/doc2graph/venv/lib/python3.5/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 10
iteration 20
iteration 30
iteration 40
iteration 50
iteration 60
iteration 70
iteration 80
iteration 90
iteration 100
iteration 110
iteration 120
iteration 130
iteration 140
iteration 150
iteration 160
iteration 170
iteration 180
iteration 190
iteration 200
iteration 210
iteration 220
iteration 230
iteration 240
iteration 250
iteration 260
iteration 270
iteration 280
iteration 290
iteration 300
iteration 310
iteration 320
iteration 330
iteration 340
iteration 350
iteration 360
iteration 370
iteration 380
iteration 390
iteration 400
iteration 410
iteration 420
iteration 430
iteration 440
iteration 450
iteration 460
iteration 470
iteration 480
iteration 490


In [66]:
model.docvecs.most_similar("1")

[('430', 0.8913657665252686),
 ('138', 0.8643275499343872),
 ('400', 0.8520273566246033),
 ('58', 0.8425953388214111),
 ('398', 0.8381558656692505),
 ('257', 0.8364518284797668),
 ('196', 0.835493803024292),
 ('156', 0.8344122171401978),
 ('223', 0.8268957138061523),
 ('433', 0.8143468499183655)]

In [78]:
print(1,tweets[1].text)
print(430,tweets[430].text)
print(58,tweets[58].text)
print(61,tweets[61].text) # <- BURASI FARKLI

1 Kibir, bele bağlanmış taş gibidir. Onunla ne yüzülür ne de uçulur. (Hacı Bayram-ı Veli)
430 Ne güzel ümmetin olmak..Ne güzel seni görmeden sevmek Ya Resulallah (S.A.V.) Kandilimiz mübarek olsun.. http://t.co/KJg7HYrjJ6
58  Gel de birbirimizin kıymetini bilelim. Çünkü ansızın ayrılacağız birbirimizden.  // Mevlana
61 Önce Trabzonspor,  sonra Fenerbahçe şimdi ise Beşiktaş ! haydı hayırlısı...
Şaka gibi...
